In [2]:
import pandas as pd
import numpy as np
import urllib.request
import requests
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB, MultinomialNB 
from sklearn.linear_model import Perceptron
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import re
from pymystem3 import Mystem
from stop_words import get_stop_words
from transformers import pipeline
import json
import os
from os.path import join

In [24]:
data_test_filename = join(os.environ["HOME"], "Datasets/llm_antonyms.csv")
df_test = pd.read_csv(data_test_filename, sep=',', index_col=False)
df_test

,rating,text,req_id,text_converted
0,5.0,Приятное место. Обедали компанией в субботу. М...,d7qdtrqejohtsbou5vrt,НевдохноВительное место. Ужинали в одиночку в ...
1,4.0,"Хорошее кафе, есть для детей игровой уголок и ...",d7qc1e1b9js18oqgk1sl,"Плохое кафе, нет для детей игрового уголка и д..."
2,4.0,\n4 с минусом. Туалет это дичь просто. Если ты...,d7qe7m906d2iugace0oa,4 с плюсом. Туалет — это для кого как… Если вы...
3,5.0,Прекрасное заведение и атмосфера. Все было оче...,d7qenpbd3ju94apa5kcl,Обычное заведение и плохая атмосфера. Всё было...
4,5.0,"Вежливый персонал, быстрая скорость отдачи зак...",d7qkb7ovkblv3m03ot78,"Грубый персонал, медленная скорость отдачи зак..."
...,...,...,...,...
4995,3.0,Посмотрели на супер отзывы и пришли сюда за Ку...,d7q4fgje6f5aehqvu1p2,Прочитали негативные отзывы и зашли сюда в пои...
4996,5.0,Любимое место уже на протяжении многих лет ❤️ ...,d7quqlv85ppkv8vs7hsr,Нелюбимое место на протяжении многих лет ☹️ ча...
4997,5.0,Очень вкусное и свежее пиво по замечательной ц...,d7qja44vtt4jiocquitj,Невкусное и несвежее пиво за неоправданно высо...
4998,5.0,"Место не дешевое, но очень уютное. Прекрасный ...",d7qo97i695n56i6n7i8e,"Место недешёвое, зато очень уютное. Скромный п..."


In [30]:
df_feltered = df_test[df_test['text_converted'].str.contains('[а-яА-Я]', na=False)]
df_test = df_feltered.copy()

,rating,text,req_id,text_converted
0,5.0,Приятное место. Обедали компанией в субботу. М...,d7qdtrqejohtsbou5vrt,НевдохноВительное место. Ужинали в одиночку в ...
1,4.0,"Хорошее кафе, есть для детей игровой уголок и ...",d7qc1e1b9js18oqgk1sl,"Плохое кафе, нет для детей игрового уголка и д..."
2,4.0,\n4 с минусом. Туалет это дичь просто. Если ты...,d7qe7m906d2iugace0oa,4 с плюсом. Туалет — это для кого как… Если вы...
3,5.0,Прекрасное заведение и атмосфера. Все было оче...,d7qenpbd3ju94apa5kcl,Обычное заведение и плохая атмосфера. Всё было...
4,5.0,"Вежливый персонал, быстрая скорость отдачи зак...",d7qkb7ovkblv3m03ot78,"Грубый персонал, медленная скорость отдачи зак..."
...,...,...,...,...
4995,3.0,Посмотрели на супер отзывы и пришли сюда за Ку...,d7q4fgje6f5aehqvu1p2,Прочитали негативные отзывы и зашли сюда в пои...
4996,5.0,Любимое место уже на протяжении многих лет ❤️ ...,d7quqlv85ppkv8vs7hsr,Нелюбимое место на протяжении многих лет ☹️ ча...
4997,5.0,Очень вкусное и свежее пиво по замечательной ц...,d7qja44vtt4jiocquitj,Невкусное и несвежее пиво за неоправданно высо...
4998,5.0,"Место не дешевое, но очень уютное. Прекрасный ...",d7qo97i695n56i6n7i8e,"Место недешёвое, зато очень уютное. Скромный п..."


In [70]:
data_train_filename = join(os.environ["HOME"], "Datasets/yandex_data_train.csv")
df_train = pd.read_csv(data_train_filename, sep=',', index_col=False, usecols=lambda column: column not in ['Unnamed: 0'])
df_train.head(500)

,rating,text
0,4.0,"Ресторан в центре Москвы, в замечательном мест..."
1,4.0,"Кухня не плохая, довольно таки вкусно. Но для ..."
2,5.0,"Отличный ресторан, попробовали очень много блю..."
3,4.0,"Бизнес-встречу здесь планировать провально , а..."
4,5.0,Завтракали в этом ресторане при гостинице. Все...
...,...,...
495,3.0,Еда была не вкусная. Тыквенный суп слишком кис...
496,5.0,Кафе прекрасное. Средний чек на двоих(2000 с ч...
497,5.0,"Просто волшебно, невероятно, бесподобно! \nСка..."
498,5.0,В данном ресторане люблю бывать. Нравится все:...


In [32]:
stop_words = get_stop_words('russian')
def remove_russian_stop_words(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

mystem = Mystem()
# Function to apply stemming to Russian text
def stem_russian_text(text):
    tokens = mystem.lemmatize(text)
    return ''.join(tokens)


def prepare(data, col='text'):
    # Step 1: Remove punctuation from the text column
    data[col] = data[col].apply(lambda x: re.sub(r'[^\w\s]', ' ', x))

    # Step 2: Remove stop words from the text column
    data[col] = data[col].apply(remove_russian_stop_words)
    data[col] = data[col].apply(stem_russian_text)


In [33]:
prepare(df_test)
prepare(df_test, 'text_converted')
prepare(df_train)

/var/folders/m0/w4ll3q093gn3rx7bm5rtb2wh0000gn/T/ipykernel_5971/3926272427.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = data[col].apply(lambda x: re.sub(r'[^\w\s]', ' ', x))
/var/folders/m0/w4ll3q093gn3rx7bm5rtb2wh0000gn/T/ipykernel_5971/3926272427.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = data[col].apply(remove_russian_stop_words)
/var/folders/m0/w4ll3q093gn3rx7bm5rtb2wh0000gn/T/ipykernel_5971/3926272427.py:20: SettingWithCopyWarning: 
A value is trying to be set 

In [35]:
from nltk.tokenize import word_tokenize

# Define a function to swap adjectives with their antonyms in a text
def swap_adjectives(text):
    tokens = word_tokenize(text, language='russian')
    swapped_text = []
    for token in tokens:
        if token in ant:
            swapped_text.append(ant[token])
        elif token in ant_adv:
            swapped_text.append(ant_adv[token])
        else:
            swapped_text.append(token)
    return ' '.join(swapped_text)

# Example usage
text = "Большой красивый дом стоит на холме."
swapped_text = swap_adjectives(text)
print(swapped_text)

Большой непривлекательным дом стоит на холме .


In [36]:
df_test["text"]

0        приятный место обедать компания суббота место ...
1        хороший кафе ребенок игровой уголок детский ме...
2        n4 минус туалет дичь мужчина вставать унитаз п...
3        прекрасный заведение атмосфера вкусно божестве...
4        вежливый персонал быстрый скорость отдача зака...
                               ...                        
27655    заказывать доставка привозить упаковывать быст...
27656    доброжелательный место отличный интерьер музык...
27657    хорший ресторанчик набережнрй город герой прек...
27658    вкусный кафе относительно демократичный цена п...
27659    отличный кафе кухня супер вкусно быстро владик...
Name: text, Length: 27660, dtype: object

In [37]:
df_test["text_ant"] = df_test['text'].apply(swap_adjectives)

In [34]:
df_all = pd.concat([df_train['text'], df_test['text'], df_test['text_converted']])
df_all

0       ресторан центр москва замечательный место хоро...
1       кухня плохой таки вкусно обстановка подвал дор...
2       отличный ресторан попробовать блюдо вкусно пер...
3       бизнес встреча планировать провальный проводит...
4       завтракать ресторан гостиница невероятно вкусн...
                              ...                        
4995    прочитывать негативный отзыв зайти сюда поиск ...
4996    нелюбимый место протяжение многий огорчать сер...
4997    невкусный несвежий пиво неоправданно высокий ц...
4998    место недешевый уютный скромный персонал напит...
4999    посещать место настолько понравиться передумыв...
Length: 74507, dtype: object

In [35]:
vectorizer = TfidfVectorizer()
vectorizer.fit(df_all)

TfidfVectorizer()

,rating,text
0,4.0,"Ресторан в центре Москвы, в замечательном мест..."
1,4.0,"Кухня не плохая, довольно таки вкусно. Но для ..."
2,5.0,"Отличный ресторан, попробовали очень много блю..."
3,4.0,"Бизнес-встречу здесь планировать провально , а..."
4,5.0,Завтракали в этом ресторане при гостинице. Все...
...,...,...
64532,4.0,"Чтобы получить отзыв 5,0 нужно очень постарат..."
64533,5.0,Очень уютное кафе! Была там несколько раз. Пер...
64534,3.0,"Невозможно заказать с доставкой, мало мест для..."
64535,5.0,"Доброжелательный персонал. Вкусная еда, хороша..."


In [75]:
X_train = vectorizer.transform(df_train['text'])
y_train = df_train['rating'].apply(lambda x: x > 4)

X_test = vectorizer.transform(df_test['text'])
X_drift_tesst = vectorizer.transform(df_test['text_converted'])
y_test = df_test['rating'].apply(lambda x: x > 4)

In [76]:
clf = MultinomialNB(alpha=0.2)
clf.fit(X_train, y_train)

MultinomialNB(alpha=0.2)

In [77]:
# NO DRIFT
y_pred = clf.predict(X_test)

# Step 6: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred, target_names=["Not 5 star", "5 star"]))

Accuracy: 0.8720160481444333
              precision    recall  f1-score   support

  Not 5 star       0.85      0.57      0.68      1199
      5 star       0.88      0.97      0.92      3786

    accuracy                           0.87      4985
   macro avg       0.86      0.77      0.80      4985
weighted avg       0.87      0.87      0.86      4985



In [78]:
#  DRIFT
y_pred = clf.predict(X_drift_tesst)

# Step 6: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred, target_names=["Not 5 star", "5 star"]))

Accuracy: 0.4164493480441324
              precision    recall  f1-score   support

  Not 5 star       0.26      0.75      0.38      1199
      5 star       0.80      0.31      0.45      3786

    accuracy                           0.42      4985
   macro avg       0.53      0.53      0.41      4985
weighted avg       0.67      0.42      0.43      4985



In [88]:
X_test_retrain, X_test_30, y_test_retrain, y_test_30 = train_test_split(X_drift_tesst, y_test, test_size=0.4, random_state=42)

In [89]:
clf.partial_fit(X_test_retrain, y_test_retrain)

MultinomialNB(alpha=0.2)

In [90]:
y_pred = clf.predict(X_test_30)

# Step 6: Evaluate the model
accuracy = accuracy_score(y_test_30, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test_30, y_pred, target_names=["Not 5 star", "5 star"]))

Accuracy: 0.8515546639919759
              precision    recall  f1-score   support

  Not 5 star       0.88      0.48      0.62       504
      5 star       0.85      0.98      0.91      1490

    accuracy                           0.85      1994
   macro avg       0.86      0.73      0.76      1994
weighted avg       0.85      0.85      0.84      1994



In [101]:
import pandas as pd
from collections import Counter
import numpy as np
from scipy.stats import ks_2samp

def calculate_word_frequencies(text_data):
    # Tokenize text and count word frequencies
    tokenized_text = [word for sentence in text_data for word in sentence.split()]
    word_freq = Counter(tokenized_text)
    return word_freq

def detect_data_drift(part1_data, part2_data):
    # Calculate word frequencies for each part
    part1_word_freq = calculate_word_frequencies(part1_data)
    part2_word_freq = calculate_word_frequencies(part2_data)

    # Extract word frequencies and sort by word
    part1_word_freq_values = np.array(list(part1_word_freq.values()))
    part2_word_freq_values = np.array(list(part2_word_freq.values()))

    # Calculate Kolmogorov-Smirnov statistic
    ks_statistic, p_value = ks_2samp(part1_word_freq_values, part2_word_freq_values)

    # Define a threshold for significant drift (you can adjust this threshold)
    threshold = 0.05

    print(p_value)
    # Check for data drift
    if p_value < threshold:
        print("Data drift detected!")
    else:
        print("No significant data drift detected.")

# Sample data (replace with your dataset)
part1_data = list(df_test["text"])
part2_data = list(df_train["text"])

# Detect data drift
detect_data_drift(list(df_train['text']), list(df_test['text']))

0.0
Data drift detected!


In [102]:
period1_text = df_train['text']
period2_text = df_test['text']

p = vectorizer.transform(period1_text)
q = vectorizer.transform(period2_text)

# Convert to dense array for Hellinger distance calculation
p = p.toarray().mean(axis=0)
q = q.toarray().mean(axis=0)
np.sqrt(np.sum((np.sqrt(p) - np.sqrt(q))**2)) / np.sqrt(2)

1.241055368765137